### No need of the patches (from all the tissue area) split by CLAM
### Need the correspondent WSI instead
### For multi-thread, better to use Spyder (also seems faster) than Jupyter. Jupyter will stop all the graphic views (e.g. webpages) if exceed memory.

# New Section

In [ ]:
import os
import re
from PIL import Image
import h5py
import numpy as np
import math
# import sys
# !conda install --yes --prefix {sys.prefix} ipywidgets # install in the current conda environment
# !jupyter nbextension enable --py widgetsnbextension 
# Then restart jupyter
from tqdm.notebook import tqdm
## for Spyder
#from functools import partial
#tqdm = partial(tqdm, position=0, leave=True)
# from numba import jit, njit, cuda
# !sudo apt-get install openslide-tools
# !sudo apt-get install python-openslide
# !pip install openslide-python
# import openslide
from IPython.display import clear_output # no idea why sometime it will freeze your PC when trying to reconnect automatically

In [ ]:
# path_tiles_qp = "/media/visiopharm5/WDRed(backup)/qupath_mondor/tiles_20x_256"
# path_slide = "/media/visiopharm5/WDGold/deeplearning/Hepatocarcinomes/slides_annotations_hammamatsu"
# path_patch_anno = "/media/visiopharm5/WDRed(backup)/clam_extension/results/patches_mondor_tumor"

from google.colab import drive
drive.mount('/content/drive')
path_tiles_qp = "/content/drive/My Drive/encoded" # tiles_20x_256
path_slide = "/content/drive/My Drive/TCGA_HCC_diagnostic_slides"
path_patch_anno = "/content/drive/My Drive/patches_mondor_tumor"

Mounted at /content/drive


In [ ]:
print("!")

!


In [ ]:
# Iterate the whole directory
slide_id = []

start = 0
stop = 50

for root, dirs, files in os.walk(path_tiles_qp):
#     display(dirs)
    slide_id =  sorted(dirs) # sort the dirs to ensure the order, then for indexing
    break
print(len(slide_id))

slide_id = slide_id[start: stop]
display(slide_id)

27


['HMNT0111_bis - 2017-06-17 20.43.09',
 'HMNT0123_bis - 2017-06-04 12.12.57',
 'HMNT0124 - 2017-06-05 04.23.20',
 'HMNT0129 - 2017-06-17 16.25.03',
 'HMNT0129_bis - 2017-06-17 17.15.39',
 'HMNT0133 - 2017-06-09 01.59.50',
 'HMNT0183 - 2017-07-15 01.37.30',
 'HMNT0188 - 2017-06-18 11.05.02',
 'HMNT0188_bis - 2017-06-18 11.49.33',
 'HMNT0198 - 2017-06-15 20.40.51',
 'HMNT0198_bis - 2017-06-15 21.26.10',
 'HMNT0365_bis - 2017-05-31 08.50.25',
 'HMNT0409_bis - 2017-07-16 06.00.19',
 'HMNT0454 - 2017-06-04 23.33.35',
 'HMNT0455 - 2017-06-16 16.46.38',
 'HMNT0455_bis - 2017-06-16 17.36.32',
 'HMNT0457 - 2017-06-16 20.38.26',
 'HMNT0465 - 2017-06-05 07.39.31',
 'HMNT0478_bis - 2017-06-05 09.59.58',
 'HMNT0484 - 2017-06-18 12.39.04',
 'HMNT0484_bis - 2017-06-18 13.25.02',
 'HMNT0524_bis - 2017-06-19 10.31.17',
 'HMNT0563_bis - 2017-06-13 15.58.10',
 'HMNT0569 - 2017-06-14 02.26.40',
 'HMNT0775_bis - 2017-06-11 01.56.19',
 'HMNT1383_bis - 2017-06-15 08.27.34',
 'HMNT1750 - 2017-07-08 00.44.16']

In [ ]:
tile_id = []
mask_id =  []

for name in tqdm(slide_id):
    tmp_tile = []
    tmp_mask = []
    for root, dirs, files in os.walk(os.path.join(path_tiles_qp, name)):
        if len(files) != 0:
            print(1)
            for file in tqdm(files):
                if file.endswith(".tif"):
                    tmp_tile.append(file)
                elif file.endswith(".png"):
                    tmp_mask.append(file)
            tile_id.append(tmp_tile)
            mask_id.append(tmp_mask)

1



1



1



1


KeyboardInterrupt: ignored

In [ ]:
for i in range(len(slide_id)):
    print(tile_id[i][0])

HMNT0111_bis - 2017-06-17 20.43.09 [x=54784,y=97280,w=512,h=512].tif
HMNT0123_bis - 2017-06-04 12.12.57 [x=71168,y=79872,w=512,h=512].tif
HMNT0129 - 2017-06-17 16.25.03 [x=23552,y=86016,w=512,h=512].tif
HMNT0129_bis - 2017-06-17 17.15.39 [x=75776,y=95744,w=512,h=512].tif


IndexError: ignored

In [ ]:
for i in range(len(slide_id)):
    if len(tile_id[i]) != len(mask_id[i]):
        print("wrong")

In [ ]:
# Necessary, sometime data loading from google cloud be a total mess (strange).
for i in range(len(slide_id)):
    if tile_id[i][0].startswith(slide_id[i]) and mask_id[i][0].startswith(slide_id[i]):
        pass
    else:
        raise Exception("os.walk() returns arbitrary orders.")

In [ ]:
# ## Specific list to process
# list_proc = ["TCGA-G3-AAV6-01Z-00-DX1.F4D85E6A-7F71-47D4-BEB6-984FDD218A9F"]
# slide_id = []
# tile_id = []
# mask_id =  []
# for root, dirs, files in os.walk(path_tiles_qp):
#     for name in dirs:
#         if name in list_proc:
#             slide_id.append(name)
#     tmp_tile = []
#     tmp_mask = []
#     if root.split("/")[-1] in list_proc:
#         for file in files:
#             if file.endswith(".tif"):
#                 tmp_tile.append(file)
#             elif file.endswith(".png"):
#                 tmp_mask.append(file)
#         tile_id.append(tmp_tile)
#         mask_id.append(tmp_mask)

In [ ]:
# re.split("=|,",'TCGA-G3-AAV6-01Z-00-DX1.F4D85E6A-7F71-47D4-BEB6-984FDD218A9F [x=10240,y=19968,w=512,h=512].tif')

In [ ]:
# print(mask_id[0:3])

In [ ]:
import pandas as pd
import re
slide_info = pd.read_csv('/content/drive/My Drive/slide_info.csv')
print(slide_info.shape)
display(slide_info.head(5))

(756, 4)


,Unnamed: 0,downsampled_level_dim,level_dim,wsi_name
0,0,"{59520, 41216}","{59520, 41216}",HMNT0001
1,1,"{53760, 30464}","{53760, 30464}",HMNT0001_bis - 2017-05-31 15.03.09
2,2,"{111360, 49664}","{111360, 49664}",HMNT0003 - 2017-05-29 15.41.33
3,3,"{51840, 49664}","{51840, 49664}",HMNT0003_bis
4,4,"{57536, 50048}","{115072, 100096}",HMNT0004 - 2017-07-08 21.07.19


In [ ]:
# Sometimes you can only see the new file using the folder icon just on the left bar, neither on the web browser, nor by other Colab notebook.
# def main(s:int=0):
  # for i in tqdm(range(s, len(tile_id))): # for each slide / bag
for i in tqdm(range(len(tile_id))): # for each slide / bag
    print("\n"+str(i)+": "+slide_id[i])
    if os.path.isfile(os.path.join(path_patch_anno, slide_id[i]+'.h5')):
        print("Already existed. Source folder is" + path_tiles_qp.split("/")[-1] + ".")
    else:
        with h5py.File(os.path.join(path_patch_anno, slide_id[i]+'.h5'), "w") as f: # create a h5 file for each bag
            init = True

            for j in tqdm(range(len(tile_id[i]))): # for each patch
                mask = np.asarray(Image.open(os.path.join(path_tiles_qp, slide_id[i], tile_id[i][j]
                                                          .replace(".tif","-labelled.png"))))
                if sum(sum(mask==1)) >= 32768: # only annotated higher than 50% will be considered
                    # coords
                    x = int(re.split("=|,",tile_id[i][j])[1])//2
                    y = int(re.split("=|,",tile_id[i][j])[3])//2
                    if init:
                        coords = [x, y]
                    else:
                        coords = np.vstack((coords, [x, y]))

                    # imgs
                    if init:
                        imgs = np.asarray(Image.open(os.path.join(path_tiles_qp, slide_id[i], tile_id[i][j])))
                        imgs = imgs[np.newaxis, :, :, :]
                        init = False
                    else:
                        imgs = np.vstack((imgs, np.asarray(Image.open(os.path.join(path_tiles_qp, slide_id[i], 
                                                                                    tile_id[i][j])))[np.newaxis, :, :, :]))
                            
            coords = coords.astype(np.int32) # convert from string to int32
            f.create_dataset("coords", data=coords)
            imgs = f.create_dataset("imgs", data=imgs)


                    
            # coords have no attributes
            imgs.attrs["downsample"] = np.asarray([np.float64(1), np.float(1)])
            imgs.attrs["downsampled_level_dim"] = np.asarray([np.int64(re.split('{|}|,', slide_info[slide_info['wsi_name']==slide_id[i]]['downsampled_level_dim'].values[0])[1]), np.int64(re.split('{|}|,', slide_info[slide_info['wsi_name']==slide_id[i]]['downsampled_level_dim'].values[0])[2])])
            imgs.attrs["level_dim"] = np.asarray([np.int64(re.split('{|}|,', slide_info[slide_info['wsi_name']==slide_id[i]]['level_dim'].values[0])[1]), np.int64(re.split('{|}|,', slide_info[slide_info['wsi_name']==slide_id[i]]['level_dim'].values[0])[2])])
            imgs.attrs["patch_level"] = np.int64(0)
            imgs.attrs["wsi_name"] = slide_id[i]



0: HMNT0569 - 2017-06-14 02.26.40


In [ ]:
print('!')

!
